In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import scipy.io as scio
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.animation import FuncAnimation
path = r"D:\GCN-learning\Paper-Chi\pygcn-master\data\cora\ER.mat"
data = scio.loadmat(path)
features = data['tz']
adj = data['aw']
labels = data['label']

In [2]:
res =[]
for time in range(100,1000,100):
    df=pd.DataFrame(data['res'+str(time)])
    df.rename(columns={0:str(time)}, inplace = True)
    res.append(df)
df=pd.concat(res,axis=1)
df

,100,200,300,400,500,600,700,800,900
0,0,0,64,361,841,1600,2704,4356,10201
1,0,0,1,144,484,1089,2025,3481,8836
2,0,0,0,49,289,784,1600,2916,7921
3,0,0,0,0,0,0,0,196,2401
4,0,0,0,64,324,841,1681,3025,8100
...,...,...,...,...,...,...,...,...,...
995,16,484,1225,2116,3136,4489,6241,8649,16384
996,0,9,256,729,1369,2304,3600,5476,11881
997,1681,3481,5184,6889,8649,10816,13456,16900,27225
998,3844,6400,8649,10816,12996,15625,18769,22801,34596


In [7]:
df_diff = df.diff(axis=1)
# 计算每一列的最大值
max_values = df_diff.max()

max_values

100        NaN
200     3780.0
300     3133.0
400     2915.0
500     2860.0
600     3377.0
700     3960.0
800     4984.0
900    14175.0
dtype: float64

这意味着每个时间片有5000s




In [13]:
# 初始化一个空的字典来存储每个节点的第一个正数时刻
infection_time = np.zeros(len(df.index))

# 遍历DataFrame的每一行
for index, row in df.iterrows():
    # 找到第一个大于0的元素的索引
    first_pos_index = row.gt(0).idxmax()
    # 如果这一行全是0，则first_pos_index会是第一个列名，需要检查这个值是否真的大于0
    if row[first_pos_index] > 0:
        # 将节点的索引和对应的第一个正数时刻存储到字典中
        infection_time[index] = (int(first_pos_index)/100-1)*5000+df[first_pos_index][index]
    else:
        # 如果这一行全是0，则记录为None或者其他标记值
        infection_time[index] = -1

# 打印结果
infection_time

array([ 1.0064e+04,  1.0001e+04,  1.5049e+04,  3.5196e+04,  1.5064e+04,
        4.0841e+04,  3.5025e+04,  3.0036e+04,  3.5196e+04,  4.6240e+03,
        2.5081e+04,  4.0729e+04,  3.0144e+04,  1.0016e+04,  1.0009e+04,
        4.0961e+04,  1.0025e+04,  2.0036e+04,  2.0025e+04,  5.0090e+03,
       -1.0000e+00, -1.0000e+00, -1.0000e+00,  1.0000e+02,  3.6100e+02,
        2.0100e+04, -1.0000e+00, -1.0000e+00,  2.5081e+04,  1.5025e+04,
        1.0064e+04,  2.0025e+04,  4.0121e+04,  1.0081e+04,  1.0049e+04,
        1.0100e+04,  3.5144e+04,  1.0100e+04,  3.0025e+04,  3.0009e+04,
        1.2960e+03, -1.0000e+00,  5.2890e+03,  3.0081e+04, -1.0000e+00,
        5.0810e+03,  3.5036e+04, -1.0000e+00,  2.0016e+04,  2.0036e+04,
        2.0016e+04,  4.0081e+04,  3.0081e+04, -1.0000e+00,  3.0036e+04,
        4.0081e+04,  2.2500e+02,  4.0625e+04,  2.5121e+04,  2.0081e+04,
        3.5169e+04,  4.0576e+04, -1.0000e+00,  9.0000e+00,  2.5009e+04,
       -1.0000e+00,  2.0049e+04,  3.0081e+04,  3.0009e+04,  4.05

In [16]:
adj

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [29]:
time_set = set(infection_time)

events = []

for time in time_set:
    indexes = np.where(infection_time == time)[0]
    for v in indexes:
        neighbors = np.where(adj[v] > 0)[0]
        for u in neighbors:
            if infection_time[u]<time and infection_time[u]!=-1:
                # events.append((u,v,time,1,time-infection_time[u]))
                events.append((u,v,time,1,1))

print(f"The network has {len(events)} events")

The network has 3189 events


In [30]:
columns = ["user_id", "item_id", "timestamp", "state", "feature"]
events_df = pd.DataFrame(events, columns=columns)
events_df = events_df.sort_values(by="timestamp")
events_df

,user_id,item_id,timestamp,state,feature
86,417,995,16.0,1,1
95,455,845,25.0,1,1
94,92,845,25.0,1,1
189,63,263,36.0,1,1
190,394,716,36.0,1,1
...,...,...,...,...,...
1531,93,868,41225.0,1,1
1532,107,868,41225.0,1,1
1554,851,970,41225.0,1,1
1526,334,856,41225.0,1,1


In [26]:
events_df.to_csv(r"D:\GCN-learning\tgn\data\rumor.csv", index=False)